In [ ]:
import sys
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from sympy.utilities.lambdify import lambdify

plt.style.use('fivethirtyeight')
sp.interactive.init_printing(use_latex='mathjax')